# Unit test

We test our logic. not the resources we use. That means that if, for instace we use a DB we don't care about the performance of DB. <br>
So, we mock the DB. So ask yourself this:
* If the DB is mocked, do we have any logic to test? if not - don't test it. So if all you have is a wrapper to a write operation - you have nothing to test.
* Do you need to mock the DB operation? genrally the answer is no! not during unit tests. So doing something like using moto to mock dynamodb and creating a fake Table with data it's not a good idea. It's A LOT work. it's hard to maintain.



## Data layer unit tests
let's take a look at this piece of code:
 ```python
def __init__(self):
    dynamodb = boto3.resource('dynamodb', **aws_params)
    self.table = dynamodb.Table(TABLE_NAME)


def insert_record(self, record):
   if not isinstance(record, self.model):
       raise Exception('Record model failed validation')

   item = record.dict()
   item["PK"] = self.get_key(tenant=item["tenant_id"])
   item["SK"] = self.get_key(vendor=item["vendor"], app_id=item["app_id"], id=item["installation_id"])
   
   response = self.table.put_item(
       Item=item
   )
   if self._verify_response(response):
       return record.dict()
   else:
       return {}
 ```

### How would you test this?
* Should you mock the DB and:
  * Use moto to create a fake DB? (NO)
  * Create your own mock DB implementation? (NO)
  * mock boto3, boto3.resource, Table and put_item ? (YES)
  

#### Let's do it